<a href="https://colab.research.google.com/github/erksubasi/transformers/blob/main/AskTo_CFALevel1_Curriculum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install transformers
!pip install InstructorEmbedding

In [1]:
!pip install -qqq langchain sentence_transformers faiss-cpu huggingface_hub
import os
from langchain.embeddings import HuggingFaceEmbeddings, HuggingFaceInstructEmbeddings

from langchain.vectorstores.faiss import FAISS
from huggingface_hub import snapshot_download


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.2 MB/s eta 0:00:00


In [3]:
cache_dir="cfa_level_1_cache"
vectorstore = snapshot_download(repo_id="nickmuchi/CFA_Level_1_Text_Embeddings",
                                repo_type="dataset",
                                revision="main",
                                #allow_patterns=f"books/{book}/*", # to download only the one book
                                cache_dir=cache_dir,
                                )


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
target_dir = "CFA_Level_1" #f"cfa/cfa_level_1"

In [7]:
vectorstore

'cfa_level_1_cache/datasets--nickmuchi--CFA_Level_1_Text_Embeddings/snapshots/ca947c50163da32281216ebe1cf081f9c352406b'

In [10]:
for root, dirs, files in os.walk(cache_dir):
    print(root, dirs)
    # Check if the target directory is in the list of directories
    if target_dir in dirs:
        # Get the full path of the target directory
        target_path = os.path.join(root, target_dir)

cfa_level_1_cache ['datasets--nickmuchi--CFA_Level_1_Text_Embeddings']
cfa_level_1_cache/datasets--nickmuchi--CFA_Level_1_Text_Embeddings ['blobs', 'refs', 'snapshots']
cfa_level_1_cache/datasets--nickmuchi--CFA_Level_1_Text_Embeddings/blobs []
cfa_level_1_cache/datasets--nickmuchi--CFA_Level_1_Text_Embeddings/refs []
cfa_level_1_cache/datasets--nickmuchi--CFA_Level_1_Text_Embeddings/snapshots ['ca947c50163da32281216ebe1cf081f9c352406b']
cfa_level_1_cache/datasets--nickmuchi--CFA_Level_1_Text_Embeddings/snapshots/ca947c50163da32281216ebe1cf081f9c352406b ['CFA_Level_1']
cfa_level_1_cache/datasets--nickmuchi--CFA_Level_1_Text_Embeddings/snapshots/ca947c50163da32281216ebe1cf081f9c352406b/CFA_Level_1 []


In [11]:
target_path

'cfa_level_1_cache/datasets--nickmuchi--CFA_Level_1_Text_Embeddings/snapshots/ca947c50163da32281216ebe1cf081f9c352406b/CFA_Level_1'

In [31]:
embed_instruction = "Represent the financial paragraph for document retrieval: "
query_instruction = "Represent the question for retrieving supporting documents: "

model_sbert = "sentence-transformers/all-mpnet-base-v2"
model_sbertqa = "sentence-transformers/multi-qa-mpnet-base-dot-v1"

model_emb = HuggingFaceEmbeddings(model_name=model_sbertqa)

model_instr = "hkunlp/instructor-large"
instruct_emb = HuggingFaceInstructEmbeddings(model_name=model_instr,
                                             embed_instruction=embed_instruction,
                                             query_instruction=query_instruction)


load INSTRUCTOR_Transformer
max_seq_length  512


In [32]:
# load vector store to use with langchain
docsearch = FAISS.load_local(folder_path=target_path, embeddings=model_emb)



In [20]:
# similarity search
question = "How do you hedge the interest rate risk of an MBS?"
search = docsearch.similarity_search(question, k=4)

for item in search:
    print(item.page_content)
    print(f"From page: {item.metadata['source']}")
    print("---")


Solutions 525
will refinance at now-  available lower interest rates. Thus, a security backed by 
mortgages will have a shorter maturity than was anticipated when the security 
was purchased.
20 A is correct. If commercial mortgage loans are non-  recourse loans, the lender 
can only look to the income-  producing property backing the loan for interest 
and principal repayment. If there is a default, the lender looks to the proceeds 
from the sale of the property for repayment and has no recourse against the 
borrower for any unpaid mortgage loan balance. Call protection and prepay -
ment penalty points protect against prepayment risk.
21 A is correct. With CMBS, investors have considerable call protection. An inves -
tor in a RMBS is exposed to considerable prepayment risk, but with CMBS, call 
protection is available to the investor at the structure and loan level. The call 
protection results in CMBS trading in the market more like a corporate bond
From page: Book: CFA_Equity_Bonds,

In [24]:
# similarity search
question = "List best performing asset classes in an inflationary environment?"
search = docsearch.similarity_search(question, k=3)

for item in search:
    print(item.page_content)
    print(f"From page: {item.metadata['source']}")
    print("---")


Fixed income securities
Inflation-  linked bonds 7%
Government bonds 10
Corporate bonds 23
Total fixed income securities 40%
Total 100%
1 Discuss the way the asset classes have been defined.
2 How does the asset allocation relate to the pension fund’s ambition to pay 
out a real (i.e., inflation-  adjusted) pension?
4 ABP defines an asset class category called “real assets, ” which contains asset classes (not fixed-  income 
securities) that are expected to perform well in times of inflation, but considered a major risk. The use of 
the term “real assets” differs from the use elsewhere in the CFA curriculum. The term “innovation” refers 
to alternative investments that are relatively new to ABP’s portfolio. An example of such an investment 
is music rights. “Infrastructure” is investments in non-  public equity of infrastructure projects such as 
toll roads. Inflation-  linked bonds are fixed-  income securities of which the payout depends on a measure
From page: Book: CFA_Corp_Fi_Port

In [33]:
# similarity search
question = "List best performing asset classes in an inflationary environment?"
search = docsearch.similarity_search(question, k=3)

for item in search:
    print(item.page_content)
    print(f"From page: {item.metadata['source']}")
    print("---")


Fixed income securities
Inflation-  linked bonds 7%
Government bonds 10
Corporate bonds 23
Total fixed income securities 40%
Total 100%
1 Discuss the way the asset classes have been defined.
2 How does the asset allocation relate to the pension fund’s ambition to pay 
out a real (i.e., inflation-  adjusted) pension?
4 ABP defines an asset class category called “real assets, ” which contains asset classes (not fixed-  income 
securities) that are expected to perform well in times of inflation, but considered a major risk. The use of 
the term “real assets” differs from the use elsewhere in the CFA curriculum. The term “innovation” refers 
to alternative investments that are relatively new to ABP’s portfolio. An example of such an investment 
is music rights. “Infrastructure” is investments in non-  public equity of infrastructure projects such as 
toll roads. Inflation-  linked bonds are fixed-  income securities of which the payout depends on a measure
From page: Book: CFA_Corp_Fi_Port